In [1]:
import json
import sqlite3
from pymystem3 import Mystem
mystem_analyzer = Mystem()
import re

В Телеграме можно скачать переписку в формате json.

In [2]:
with open('result.json', 'r') as f:
    data = json.load(f)

In [3]:
with open('ru_stopwords.txt', 'r') as file:
    stopwords = file.read().split('\n')

Теперь положим это в базу данных.

База состоит из двух таблиц: в одной хранятся сообщения в лемматизиованном виде и в лемматизированном виде без стоп-слов, во второй хранится информация об отправителе и времени отправки сообщения.

In [4]:
conn = sqlite3.connect('oleg.db')
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS meta 
(id int PRIMARY KEY, 
sender text, 
date text, 
time text, 
datetime text)
""")

conn.commit()

cur.execute("""
CREATE TABLE IF NOT EXISTS texts 
(id int PRIMARY KEY, 
text text, 
lemmatized text, 
no_stopwords text)
""")

conn.commit()

In [5]:
def get_lemmatized(text):
    
    if type(text) == list:
        return '', ''
    text = mystem_analyzer.lemmatize(text)
    lemmatized = []
    no_stopwords = []
    
    for word in text:
        if re.findall('\w-\w-\w|\w-\w|\w', word):
            lemmatized.append(word)
            if word not in stopwords:
                no_stopwords.append(word)
            
    return ' '.join(lemmatized), ' '.join(no_stopwords)

In [7]:
for message in data['messages']:
    msg_id = message['id']
    try:
        if message['from'] == 'Valeria Buntyakova':
            sender = 'Val'
        else:
            sender = 'Oleg'
    except KeyError:
        continue
    date, time = message['date'].split('T')
    datetime = message['date']
    text = message['text']
    lemmatized, no_stopwords = get_lemmatized(text)
    if not lemmatized:
        continue
    
    cur.execute(
                '''INSERT INTO meta (id, sender, date, time, datetime)
                    VALUES (?, ?, ?, ?, ?)''',
                    (msg_id, sender, date, time, datetime)
                )
    conn.commit()
    
    cur.execute(
                '''INSERT INTO texts (id, text, lemmatized, no_stopwords)
                    VALUES (?, ?, ?, ?)''',
                    (msg_id, text, lemmatized, no_stopwords)
                )
    conn.commit()